## Доработанная первая домашняя работа для второй домашней работы :)

_Екатерина Волошина_

In [158]:
import requests
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
import random
import time
from collections import Counter
from sklearn.model_selection import train_test_split

In [117]:
from natasha import (
    Segmenter,
    MorphVocab,
    
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,
    
    PER,
    NamesExtractor,

    Doc
)

In [118]:
from fake_useragent import UserAgent
ua = UserAgent(verify_ssl=False)

In [119]:
from nltk.tokenize import WordPunctTokenizer

WPT = WordPunctTokenizer()

In [120]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

In [121]:
session = requests.session()

Функция, которая находит фильмы:

In [122]:
def find_movies():
   url = 'https://www.afisha.ru/msk/schedule_cinema/vybor-afishi/'
   req = session.get(url, headers={'User-Agent':ua.random})
   page = req.text
   soup = BeautifulSoup(page, 'html.parser')
   lists = soup.find_all('h3', {'class':'heHLK'})
   movies = []
   for i in lists:
     movie = i.find('a')['href']
     movies.append(movie)
   return movies

Скачиваем положительные отзывы:

In [123]:
def download_positive_reviews(page):
   url = f'https://afisha.ru{page}reviews/positive'
   for _ in range(5):
     response = session.get(url)
     time.sleep(random.uniform(1.1, 5.2))

   page = response.text
   soup = BeautifulSoup(page, 'html.parser')
   reviews = soup.find_all('div', class_ ='restrict-text review__text')
   posts = [post.text for post in reviews]
   return posts

Скачиваем отрицательные отзывы:

In [124]:
def download_negative_reviews(page):
   url = f'https://afisha.ru{page}reviews/negative'
   for _ in range(5):
     response = session.get(url)
     time.sleep(random.uniform(1.1, 5.2))
   page = response.text
   soup = BeautifulSoup(page, 'html.parser')
   reviews = soup.find_all('div', class_ ='restrict-text review__text')
   posts = [post.text for post in reviews]
   return posts 

Делим выборку на тестовую и тренировочную:

In [159]:
def reviews_split(pos, neg):
    pos_train, pos_test = train_test_split(pos)
    neg_train, neg_test = train_test_split(neg)
    test = {}
    for i in pos_test:
        test[i] = 1
    for i in neg_test:
        test[i] = 0
    return pos_train, neg_train, test

Анализируем отзывы, создаем множества:

In [162]:
def analyze_reviews(pos, neg, freq):
    positive = []
    negative = []
    for i in pos:
      positive.extend(text_from_the_post(i))
    for i in neg:
      negative.extend(text_from_the_post(i))
    pos_words = Counter(positive)
    neg_words = Counter(negative) 
    positive = set(positive)
    negative = set(negative)
    common_words = positive.intersection(negative)
    positive -= common_words
    negative -= common_words
    positive = [i for i in positive if pos_words[i] > freq]
    negative = [i for i in negative if neg_words[i] > freq]
    return positive, negative

Анализируем слова, включая группы:

In [163]:
def analyze_reviews_w_groups(pos, neg, freq):
    positive = []
    negative = []
    for i in pos:
      positive.extend(text_from_the_post(i))
      positive.extend(finding_groups(i))
    for i in neg:
      negative.extend(text_from_the_post(i))
      negative.extend(finding_groups(i))
    pos_words = Counter(positive)
    neg_words = Counter(negative) 
    positive = set(positive)
    negative = set(negative)
    common_words = positive.intersection(negative)
    positive -= common_words
    negative -= common_words
    positive = [i for i in positive if pos_words[i] > freq]
    negative = [i for i in negative if neg_words[i] > freq]
    return positive, negative

Функция для поиска групп (не + прилагательное, прилагательное + существительное, наречие + глагол):

In [128]:
def finding_groups(text):
  groups_list = []
  pos_tags = []
  doc = Doc(text)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for i in doc.tokens:
    i.lemmatize(morph_vocab)
  pos_tags = [(i.lemma, i.pos) for i in doc.tokens]
  n_gram_list = []
  for i in range(len(pos_tags)):
    lemma = pos_tags[i][0]
    tag = pos_tags[i][1]
    if (i + 1) != len(pos_tags):
      next_tag = pos_tags[i+1][1]
      next_lemma = pos_tags[i+1][0]
      if lemma == 'не' and next_tag == 'ADJ':
        n_gram_list.append(' '.join([lemma, next_lemma]))
      elif tag == 'ADJ' and next_tag == 'NOUN':
        n_gram_list.append(' '.join([lemma, next_lemma]))
      elif tag == 'ADV' and next_tag == 'VERB':
        n_gram_list.append(' '.join([lemma, next_lemma]))
  return n_gram_list

Лемматизируем слова из отзывов:

In [129]:
def text_from_the_post(post):
  post = post[1:] # первый символ строки - оценка в десятибалльной системе
  doc = Doc(post)
  doc.segment(segmenter)
  doc.tag_morph(morph_tagger)
  for i in doc.tokens:
    i.lemmatize(morph_vocab)
  words = [i.lemma for i in doc.tokens if i.lemma.isalpha()]
  return words

Определяем тональность:

In [130]:
def tonality(post, pos, neg):
  text = text_from_the_post(post)
  pos_count = 0
  neg_count = 0
  mark = 0
  for i in text:
    if i in pos:
      pos_count += 1
    elif i in neg:
      neg_count += 1
  if pos_count > neg_count:
      mark = 1
  elif neg_count > pos_count:
      mark = 0
  return mark

Определяем тональность, если есть группы:

In [153]:
def tonality_w_gr(post, pos, neg):
  text = text_from_the_post(post)
  text.extend(finding_groups(post))
  pos_count = 0
  neg_count = 0
  mark = 0
  for i in text:
    if i in pos:
      pos_count += 1
    elif i in neg:
      neg_count += 1
  if pos_count > neg_count:
      mark = 1
  elif neg_count > pos_count:
      mark = 0
  return mark

Ищем фильмы:

In [132]:
movies = find_movies()

Готовим множества слов из положительных и отрицательных отзывов:

In [133]:
p_words = []
n_words = []
for page in tqdm(movies):
  positive_words = download_positive_reviews(page)
  negative_words = download_negative_reviews(page)
  p_words.extend(positive_words)
  n_words.extend(negative_words)

Делим выборку:

In [160]:
p_train, n_train, test = reviews_split(p_words, n_words)

Анализируем отзывы (отдельно без и с группами):

In [180]:
pos_words, neg_words = analyze_reviews(p_train, n_train, 2)
pos_words_w_g, neg_words_w_g = analyze_reviews_w_groups(p_train, n_train, 2)

Считаем долю верных ответов:

In [181]:
accuracy = 0
accuracy_w_gr = 0
for k, v in test.items():
  mark = tonality(k, pos_words, neg_words)
  mark_w_gr = tonality_w_gr(k, pos_words_w_g, neg_words_w_g)
  if mark == v:
    accuracy += 1
  if mark_w_gr == v:
    accuracy_w_gr += 1
accuracy = accuracy/len(test)
accuracy_w_gr = accuracy_w_gr/len(test)

Сравнием долю верных ответов без и с группами в словаре:

In [182]:
accuracy

0.7272727272727273

In [183]:
accuracy_w_gr

0.7545454545454545

Как мы видим, качество повышается, хоть и ненамного.